In [11]:
import httpx
import asyncio
import nest_asyncio
from datetime import datetime, timedelta
from telegram import Bot

# Enable asyncio in a Jupyter environment
nest_asyncio.apply()

# Configuration
BOT_TOKEN = '6593706311:AAGfep182h5A1hOifip4095pVEy4Rp7kZTs'  
CHAT_ID = '@roofpocazz'
KRAKEN_API_URL = 'https://api.kraken.com/0/public/Ticker'
symbols = ['BTCUSDT', 'ETHUSDT', 'XRPUSDT', 'LTCUSDT', 'ADAUSDT', 'SOLUSDT', 'LINKUSDT', 'XMRUSDT']

In [12]:
async def get_current_price(symbol):
    async with httpx.AsyncClient() as client:
        params = {'pair': symbol}
        response = await client.get(KRAKEN_API_URL, params=params)
        
        if response.status_code == 200:
            try:
                data = response.json()
                kraken_symbol = list(data['result'].keys())[0]  # Get the first (and only) traded pair returned.
                price = data['result'][kraken_symbol]['c'][0]  # 'c' is the last trade closed array, [0] is the price
                return price
            except Exception as e:
                print('Error processing response:', e)
                print(response.content)  # Print the full response for troubleshooting
        else:
            print(f'Failed to fetch data for {symbol}. Status code: {response.status_code}')
            return None

In [13]:
async def send_telegram_message(message):
    bot = Bot(token=BOT_TOKEN)
    await bot.send_message(chat_id=CHAT_ID, text=message)

In [14]:
async def fetch_and_send_prices():
    gmt_plus_6_time = datetime.utcnow() + timedelta(hours=6)
    message = f'KRAKEN: Prices at {gmt_plus_6_time.strftime("%Y-%m-%d %H:%M:%S")} (GMT+6):\n\n'
    
    for symbol in symbols:
        price = await get_current_price(symbol)
        if price is not None:
            formatted_price = f'{float(price):.2f}'
            
            # No need to replace 'USD' with 'USDT' as Kraken uses 'USD'
            message += f'{symbol}: ${formatted_price}\n'
    
    await send_telegram_message(message)

In [15]:
# Run the asyncio event loop
async def main():
    await fetch_and_send_prices()

loop = asyncio.get_event_loop()
loop.run_until_complete(main())